In [1]:
import pandas as pd
import numpy as np
from datautils import dir_to_dataframe

feature_dir = "data/features"
raw_features = dir_to_dataframe(feature_dir)

label_dir = "data/labels"
labels = dir_to_dataframe(label_dir)

Using TensorFlow backend.


In [2]:
from datautils import preprocess_data

booking_ids, labels, aggregate_features, timeseries_features = preprocess_data(raw_features, labels)

Feature Extraction: 100%|██████████| 30/30 [00:48<00:00,  1.53s/it]


In [3]:
from datautils import make_timeseries_same_length
timeseries = make_timeseries_same_length(timeseries_features)

In [4]:
from sklearn.preprocessing import StandardScaler
aggregate = StandardScaler().fit_transform(aggregate_features)

In [5]:
from sklearn.model_selection import train_test_split

x_train_ts, x_valid_ts, y_train, y_valid = train_test_split(timeseries, labels, test_size=0.2, shuffle=True, random_state=879)
x_valid_ts, x_test_ts, y_valid, y_test = train_test_split(x_valid_ts, y_valid, test_size=0.5)

x_train_agg, x_valid_agg, y_train, y_valid = train_test_split(aggregate, labels, test_size=0.2, shuffle=True, random_state=879)
x_valid_agg, x_test_agg, y_valid, y_test = train_test_split(x_valid_agg, y_valid, test_size=0.5)

In [6]:
from keras.models import Sequential, Model, Input

from keras.layers import Dense, Dropout, Flatten, LSTM, concatenate

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True' # fix for issue described here: https://github.com/dmlc/xgboost/issues/1715

input_aggregates = Input(shape=(x_train_agg.shape[1],), name='input_aggregates')
dense_2 = Dense(64, activation='relu', name='agg_dense_1')(input_aggregates)
dense_3 = Dense(32, activation='relu', name='agg_dense_2')(dense_2)

out = Dense(1, activation='sigmoid', name='out')(dense_3)
model = Model(input_aggregates, out)

chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_aggregates (InputLayer (None, 107)               0         
_________________________________________________________________
agg_dense_1 (Dense)          (None, 64)                6912      
_________________________________________________________________
agg_dense_2 (Dense)          (None, 32)                2080      
_________________________________________________________________
out (Dense)                  (None, 1)                 33        
Total params: 9,025
Trainable params: 9,025
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.fit(x_train_agg, y_train, epochs=10, batch_size=128, callbacks=[chk], validation_data=(x_valid_agg,y_valid))

Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Train on 15985 samples, validate on 1998 samples
Epoch 1/10
15985/15985 [==============================] - 1s 85us/step - loss: 0.5346 - acc: 0.7600 - val_loss: 0.5299 - val_acc: 0.7598

Epoch 00001: val_acc improved from -inf to 0.75976, saving model to best_model.pkl
Epoch 2/10
15985/15985 [==============================] - 0s 27us/step - loss: 0.5020 - acc: 0.7753 - val_loss: 0.5254 - val_acc: 0.7598

Epoch 00002: val_acc did not improve from 0.75976
Epoch 3/10
15985/15985 [==============================] - 0s 27us/step - loss: 0.4930 - acc: 0.7781 - val_loss: 0.5279 - val_acc: 0.7593

Epoch 00003: val_acc did not improve from 0.75976
Epoch 4/10
15985/15985 [==============================] - 0s 27us/step - loss: 0.4855 - acc: 0.7796 - val_loss: 0.5266 - val_acc: 0.7608

Epoch 00004: val_acc improved from 0.75976 to 0.76076, saving model to best_model.pkl
Epoch 5/10
15985/15985 [==============================] - 1s 37us/step - loss: 0.4805 - acc: 0.7817 - val_loss: 0.5226 - val_acc: 

In [8]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, (model.predict(x_test_agg)>0.5)) #for output neuron with sigmoid activation, >0.5 predicts 1


0.7808904452226113

In [9]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, (model.predict(x_test_agg)>0.5))

0.5936968436968437

In [11]:
#save sample to test in other notebook
test_write_df = raw_features.sample(frac=0.1)
test_write_df.to_csv('data/test_features/features.csv', index=False)